## Streamlit Google Colab

In [5]:
!pip install pyngrok
!pip install streamlit
!pip install streamlit_folium
!pip install geopandas
!pip install geojson

# then restart runtime

     |████████████████████████████████| 102kB 2.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.11.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/geonode_flood_hazard_map_vector.geojson
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/geonode_flood_hazard_map_vector.dbf
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/geonode_flood_hazard_map_vector.shp
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/geonode_flood_hazard_map_vector.prj
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/geonode_flood_hazard_map_vector.shx
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/landslides_1_4326.geojson
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/landslides_1_4326.dbf
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/landslides_1_4326.prj
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/landslides_1_4326.shp
# !wget https://github.com/simonevaccari/ExtractValues/blob/main/Flood_and_Landslide_Datasets/landslides_1_4326.shx

In [ ]:
#!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/Data_Martin/geonode_flood_hazard_map.tif
#!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/Data_Martin/landslides_1_4326.shp
#!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/Data_Martin/landslides_1_4326.geojson
#!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/Data_Martin/landslides_1_4326.prj
#!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/Data_Martin/landslides_1_4326.shx
#!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/Data_Martin/landslides_1_4326.dbf

In [15]:
ls

app.py                                   landslides_1_4326.dbf
geonode_flood_hazard_map_vector.dbf      landslides_1_4326.geojson
geonode_flood_hazard_map_vector.geojson  landslides_1_4326.prj
geonode_flood_hazard_map_vector.prj      landslides_1_4326.shp
geonode_flood_hazard_map_vector.shp      landslides_1_4326.shx
geonode_flood_hazard_map_vector.shx      sample_data/


In [14]:
import geopandas as gpd
a = gpd.read_file('landslides_1_4326.shx')
a.head()

DriverError: ignored

In [10]:
%%writefile app.py

# -*- coding: utf-8 -*-
"""
Created on Wed Oct  7 16:48:04 2020

@author: SimoneVaccari
"""

# Import Libraries
import geopandas as gpd
import geojson

from shapely.geometry import Point
import folium
from streamlit_folium import folium_static
import matplotlib, matplotlib.pyplot as plt 
import numpy as np
import branca
#-------------------

# Datasets
# Landslide 
landslide_shp = gpd.read_file('landslides_1_4326.shp')
landslide_json = 'landslides_1_4326.geojson'
#landslide_shp.head()

# Flood Risk (Vectorised)
flood_shp = gpd.read_file('geonode_flood_hazard_map_vector.shp')
flood_gj = geojson.load(open('geonode_flood_hazard_map_vector.geojson')) # Import geojson file # https://stackoverflow.com/questions/42753745/how-can-i-parse-geojson-with-python
#print(flood_shp.head())

#gj_features = flood_gj['features']
#print(gj_features[0])
#-------------------

# ================================================
st.title("Extract Values")
# Folium Tutorial link: https://python-visualization.github.io/folium/quickstart.html

# Below are some points and expected results for checking
#x, y = (-61.4198, 15.396)
#x, y = (-61.37618,15.43130) # should return 2 & "NO Landslide"
#x, y = (-61.346482,15.393996) # should return 1 & "NO Landslide"

#x, y = (-61.346078,15.394703) # should return 1 & Pol_1
#x, y = (-61.419855,15.396184) # should return 3 & Pol_1771 # this is outside the AOI clip
#x, y = (-61.415726,15.399853) # should return 1 & Pol_16 
x, y = (-61.413723,15.402079) # should return 5 & Pol_16 # this is outside the AOI clip

longitude=x; latitude=y
p = Point(longitude,latitude)
#-------------------

# Show Map

# Fill polygons in Folium *** https://stackoverflow.com/questions/35516318/plot-colored-polygons-with-geodataframe-in-folium ***
colors = ['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c'] # these have been assigned to each FloodRisk category in the GeoJSON file on QGIS!!!

m = folium.Map(location=[latitude, longitude], zoom_start=11)

# Show Landslide GeoJSON to the map
folium.GeoJson(
    landslide_json,
    name='Landslide'
).add_to(m)

# Setup colormap MUST USE SAME COLORS AS QGIS GEOJSON FILE!!!!
levels = len(colors)
cmap = branca.colormap.LinearColormap(colors, vmin=1, vmax=5).to_step(levels-1)
cmap.add_to(m)

folium.GeoJson(
    flood_gj,
    style_function=lambda feature: {
        'fillColor': feature['properties']['Color'],
        'color' : feature['properties']['Color'],
        'weight' : 1,
        'fillOpacity' : 0.3,
        }
    ).add_to(m)

# add marker 
tooltip = "Selected Point"
folium.Marker(
    [latitude, longitude], tooltip=tooltip
).add_to(m)

# Enable lat-long Popup; LayerControl; Call to render Folium map in Streamlit
m.add_child(folium.LatLngPopup())
folium.LayerControl().add_to(m)

folium_static(m)
#-------------------

st.header('Extracting Results for the location:\n(Lat: ' + str(latitude) +' & Long: ' + str(longitude) + ')')

# ======= Get Value from Shapefile
landslide_code = 'No Landslide'

# From a given Point coordinates, quickly/efficiently check if it's contained in any polygons geometry, and print out the LANDSLIDE code of that polygon if so. 
# this works, but is this teh most efficient way?!? Think it can be improved with lampda / apply / map, without iterating on the whole dataframe?
for elem in landslide_shp.loc[:,'geometry']:
  if p.within(elem): 
    landslide_code = landslide_shp.loc[landslide_shp.loc[:,'geometry'] == elem]['LANDSLIDES'].values[0]
st.text('The point is contained in the polygon: '+ landslide_code)

# ======= Get Value from Raster
frisk_code = 'NAN'

# From a given Point coordinates, quickly/efficiently check if it's contained in any polygons geometry, and print out the LANDSLIDE code of that polygon if so. 
# this works, but is this teh most efficient way?!? Think it can be improved with lampda / apply / map, without iterating on the whole dataframe?
for elem in flood_shp.loc[:,'geometry']:
  if p.within(elem): 
    frisk_code = flood_shp.loc[flood_shp.loc[:,'geometry'] == elem]['FloodRisk'].values[0]

st.text('Flood risk category: ' + str(frisk_code))


Overwriting app.py


In [ ]:
#from app import * # per debuggare il codice all'interno di app.py

In [11]:
from pyngrok import ngrok
!pkill -9 streamlit && pkill -9 ngrok
!ngrok authtoken 1du08eXGWleyOyefGF6DC7Falf6_5qjQskC7sNB5TLH3K7MJ
!streamlit run app.py&>/dev/null&
!pgrep streamlit
public_url = ngrok.connect(port='8501')
public_url

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
647


'http://d9c79bfc0bb9.ngrok.io'

In [ ]:
ngrok.kill()